This script has the code to conduct a number of geospatial operation using ArcGIS Pro arcpy Python library. This is to be run in ArcGIS Pro

In [ ]:
#Importing libraries

import arcpy, os, csv
arcpy.env.overwriteOutput = True

import pandas as pd
import numpy as np


In [ ]:
# This section has code to count the number of observations inside a polygon (for example points).
# I am using it for the habitat features inside the Nicola sub-watershed polygons.
# The output is a feature class with the counts, and a CSV table with the counts.

# --- Get layers from the current map (use your actual layer names) ---
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.activeMap
polys_lyr  = m.listLayers("Nicola Sub-watershed")[0]       # or your polygon layer name
points_lyr = m.listLayers("Habitat Features")[0]   # "ObsPoints"

# (Optional) If you have a selection but want ALL polygons counted, clear it:
arcpy.management.SelectLayerByAttribute(polys_lyr, "CLEAR_SELECTION")

# --- Spatial Join: counts points per polygon; KEEP_ALL ensures 0s are kept ---
out_fc = arcpy.CreateUniqueName("polys_with_counts", arcpy.env.scratchGDB)
arcpy.analysis.SpatialJoin(
    target_features=polys_lyr,
    join_features=points_lyr,
    out_feature_class=out_fc,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",          # <-- keeps polygons with zero points
    match_option="INTERSECT"       # use "WITHIN" if you want strictly inside (no boundary)``
)

# --- Export the table to CSV ---
csv_folder = r"path_to_folder"                # must be a folder, not a GDB
arcpy.conversion.TableToTable(out_fc, csv_folder, "Habitat_feature_2025_counts.csv")

print("Done ->", out_fc)


In [ ]:
#This section counts points by category inside each sub-watershed


# --- Map + layers (change names as needed) ---
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.activeMap

polys_lyr  = m.listLayers("Nicola Sub-watershed")[0]  # polygons
points_lyr = m.listLayers("FISS_OBSPT_point_NICOLA_Final_Filter")[0]                           # points

# ID field on polygons (used to group counts)
poly_id = "ASSESS_UNI_NUM"            # <-- change if different

# Category field on points to count BY (e.g., species, type, etc.)
category_field = "SPECIES_CD" # <-- set this to the column you want

# (optional) ensure polygon selection is cleared so all zones are in output
arcpy.management.SelectLayerByAttribute(polys_lyr, "CLEAR_SELECTION")

# --- Intersect points with polygons (honors point selections) ---
pts_in_polys = arcpy.CreateUniqueName("pts_in_polys", arcpy.env.scratchGDB)
arcpy.analysis.Intersect([points_lyr, polys_lyr], pts_in_polys, "ALL", "", "POINT")

# --- Count points per polygon × category ---
out_table = arcpy.CreateUniqueName("count_by_poly_category", arcpy.env.scratchGDB)
arcpy.analysis.Statistics(
    in_table=pts_in_polys,
    out_table=out_table,
    statistics_fields=[["OBJECTID", "COUNT"]],
    case_field=[poly_id, category_field]  # group by polygon AND category
)
# output fields will include: ID_NEW, Type_Water, COUNT_OBJECTID

# --- 3) Export to CSV ---
csv_folder = r"path_to_folder"
#os.makedirs(csv_folder, exist_ok=True)
csv_name = f"point_counts_by_{category_field}.csv"
arcpy.conversion.TableToTable(out_table, csv_folder, csv_name)


In [ ]:
#This section works with line features computing the line length per group (species). It works for CFW Model data and PSF spawning lines layers.

# --- Layers already in your CURRENT map (type the layer names as they appear in Contents) ---
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.activeMap

polys_lyr   = m.listLayers("Nicola Sub-watershed")[0]    # polygons
streams_lyr = m.listLayers("Coho Spawning Model- CWF model")[0]               # lines

# ---- Polygon ID field (used to aggregate & join) ----
poly_id = "REVREPUNI"        # <-- adjust to your polygon ID field
species_field = "rearing_co"  # field that identifies species

# ---- Split stream lines by polygon boundaries ----
streams_in_polys = arcpy.CreateUniqueName("streams_in_polys", arcpy.env.scratchGDB)
arcpy.analysis.Intersect([streams_lyr, polys_lyr], streams_in_polys, "ALL", "", "LINE")

# ---- Add length to each split segment ----
sr = arcpy.Describe(streams_lyr).spatialReference  # use the layer’s SR

arcpy.management.AddGeometryAttributes(
    Input_Features=streams_in_polys,
    Geometry_Properties="LENGTH_GEODESIC",
    Length_Unit="KILOMETERS",
    Area_Unit="",
    Coordinate_System=sr
)
len_field = "LENGTH_GEO"  # field added by tool

# ---- Sum length per polygon × species ----
len_table = arcpy.CreateUniqueName("len_by_poly_species", arcpy.env.scratchGDB)
arcpy.analysis.Statistics(
    in_table=streams_in_polys,
    out_table=len_table,
    statistics_fields=[[len_field, "SUM"]],
    case_field=[poly_id, species_field]   # <-- group by polygon AND species
)

# ---- Export the polygon × species summary as CSV ----
csv_folder = r"path_to_folder"
arcpy.conversion.TableToTable(len_table, csv_folder, "cwf_stream_length_rearing_coho_C.csv")

print(f"Done. Stream length per polygon × species written to {len_table} and exported as CSV.")

In [ ]:
#This section works for area based layers like Wetlands and floodplains. We want to compute the area of polygons inside each sub-watershed, and export a CSV with the polygon ID, total area, and area of the feature inside the polygon.

# --- Layers already in your CURRENT map ---
aprx = arcpy.mp.ArcGISProject("CURRENT"); m = aprx.activeMap
zones_lyr  = m.listLayers("Nicola Sub-watershed")[0]
zone_id    = "ASSESS_UNI_NUM"                                 # polygon ID
wet_lyr    = m.listLayers("Floodplains in BC (Historical)")[0]             # areas to sum inside polygons

# --- Output CSV path ---
csv_folder = r"path_to_folder"
csv_path   = os.path.join(csv_folder, "zone_areas_floodplains.csv")  # <-- name output file here

# --- fields on the Sub-watershed layer where we will store results ---
fld_zone_area  = "AREA_ZONE"    # shortened to fit 10-char limit
fld_inner_area = "AREA_INNER"   # shortened to fit 10-char limit

# Ensure result fields exist (Double). If not, create them.
existing = {f.name.upper() for f in arcpy.ListFields(zones_lyr)}
if fld_zone_area.upper() not in existing:
    arcpy.management.AddField(zones_lyr, fld_zone_area, "DOUBLE", field_alias="Zone area (ha)")
if fld_inner_area.upper() not in existing:
    arcpy.management.AddField(zones_lyr, fld_inner_area, "DOUBLE", field_alias="Wetlands area (ha)")

# --- polygon areas (hectares, geodesic) ---
sr_zones = arcpy.Describe(zones_lyr).spatialReference
arcpy.management.CalculateGeometryAttributes(
    zones_lyr,
    geometry_property=[[fld_zone_area, "AREA_GEODESIC"]],
    area_unit="HECTARES",
    coordinate_system=sr_zones
)

# --- cut wetlands/floodplains by zones & compute area per piece (hectares) ---
wet_in_zones = arcpy.CreateUniqueName("wet_in_zones", arcpy.env.scratchGDB)
arcpy.analysis.Intersect([wet_lyr, zones_lyr], wet_in_zones, "ALL", "", "INPUT")
sr_clip = arcpy.Describe(wet_in_zones).spatialReference
arcpy.management.AddGeometryAttributes(
    Input_Features=wet_in_zones,
    Geometry_Properties="AREA_GEODESIC",
    Area_Unit="HECTARES",
    Length_Unit="",
    Coordinate_System=sr_clip
)
piece_area = "AREA_GEO"   # field added by AddGeometryAttributes (ha)

# --- sum wetlands/floodplains area per polygon ---
sum_tbl = arcpy.CreateUniqueName("wet_area_by_zone", arcpy.env.scratchGDB)
arcpy.analysis.Statistics(
    in_table=wet_in_zones,
    out_table=sum_tbl,
    statistics_fields=[[piece_area, "SUM"]],
    case_field=[zone_id]
)
sum_field = f"SUM_{piece_area}"

# ---push the totals into zones_lyr[AREA_INNER]; fill 0 when no wetlands ---
# build a dict zone_id -> sum_area
sums = {}
with arcpy.da.SearchCursor(sum_tbl, [zone_id, sum_field]) as cur:
    for zid, val in cur:
        sums[zid] = 0.0 if val is None else float(val)

with arcpy.da.UpdateCursor(zones_lyr, [zone_id, fld_inner_area]) as ucur:
    for zid, _ in ucur:
        ucur.updateRow([zid, sums.get(zid, 0.0)])

# --- write the CSV ---
os.makedirs(csv_folder, exist_ok=True)
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow([zone_id, fld_zone_area, fld_inner_area])
    with arcpy.da.SearchCursor(zones_lyr, [zone_id, fld_zone_area, fld_inner_area]) as cur:
        for zid, a_zone, a_in in cur:
            w.writerow([zid, 0.0 if a_zone is None else a_zone,
                            0.0 if a_in   is None else a_in])

print("Done.")
print(f"CSV -> {csv_path}")

In [ ]:
#After merging all the csv files with the summary of each layer per polygons (we merge using the polygon ID field), we can use the following code to normalize, get the quartiles and score each layer inside the polygons. You can run this part outisde of ArcGIS Pro as well.

In [ ]:
# Normalization first

df = pd.read_csv("path_to_master_summary_file.csv")

id_col = "ASSESS_UNI_NUM" # --- this is the polygon ID column
denom_stream = "total_stream_length"
denom_area = "polygon_total_area_km2"
area_cols = ["wetland_area_km2"] #--- columns that have area components

# Columns to EXCLUDE from "per km stream" normalization
exclude = {id_col, denom_stream, denom_area, *area_cols}

# Everything else gets divided by total_stream_length (counts, lengths, etc.)
stream_cols = [c for c in df.columns if c not in exclude]

# --- Normalize ---
# Handle divide-by-zero by using NaN (can fill with 0 if you prefer)
norm_stream = df[stream_cols].div(df[denom_stream].replace(0, np.nan), axis=0)
norm_area = df[area_cols].div(df[denom_area].replace(0, np.nan), axis=0)

# Add clear suffixes
norm_stream = norm_stream.add_suffix("_per_km_stream")
norm_area = norm_area.add_suffix("_per_km2_ws")

# Build output with ID + normalized columns
out = pd.concat([df[[id_col]], norm_stream, norm_area], axis=1)

# Optional: replace inf with NaN, or fill NaN with 0 depending on your policy
out = out.replace([np.inf, -np.inf], np.nan)
# out = out.fillna(0)

# Save
out.to_csv("C:path_to_folder/normalized_metrics.csv", index=False)

In [ ]:
#For quartiles scoring

# --- Load your normalized table ---
df = pd.read_csv(r"path_to_folder/normalized_metrics.csv")

id_col = "ASSESS_UNI_NUM"  #---polygon ID column
metric_cols = [c for c in df.columns if c != id_col]

# Choose whether to EXCLUDE zeros when computing cutpoints (useful for zero-inflated metrics)
EXCLUDE_ZEROS_FOR_QUARTILES = True

def quartiles(s: pd.Series, exclude_zeros=True):
    """Return (q25, q50, q75) for a series; optionally compute on positive values only."""
    s = s.dropna()
    if exclude_zeros:
        s = s[s > 0]
    if len(s) == 0:
        return (np.nan, np.nan, np.nan)
    return (s.quantile(0.25), s.quantile(0.50), s.quantile(0.75))

def score_with_quartiles(v, q25, q50, q75, zero_is_zero=True):
    """
    Map a value to {0,1,2,3,4} using quartiles.
    0 is reserved for literal zeros (if zero_is_zero=True).
    """
    if pd.isna(v):
        return np.nan
    if zero_is_zero and v == 0:
        return 1
    if any(pd.isna(x) for x in (q25, q50, q75)):
        return np.nan  # not enough info
    if v <= q25:
        return 2
    elif v <= q50:
        return 3
    elif v <= q75:
        return 4
    else:
        return 5

# --- Compute cutpoints per metric ---
cuts = {}
for col in metric_cols:
    q25, q50, q75 = quartiles(df[col], exclude_zeros=EXCLUDE_ZEROS_FOR_QUARTILES)
    cuts[col] = {"q25": q25, "q50": q50, "q75": q75, "n": df[col].notna().sum()}

cuts_df = pd.DataFrame(cuts).T  # for saving/inspection

#  Score every metric 1-5 ---
scores = df[[id_col]].copy()
for col in metric_cols:
    q25, q50, q75 = cuts[col]["q25"], cuts[col]["q50"], cuts[col]["q75"]
    scores[col + "_score"] = df[col].apply(lambda v: score_with_quartiles(v, q25, q50, q75, zero_is_zero=True))

# --- Save outputs ---
scores.to_csv("path_to_folderquartile_scores.csv", index=False)
cuts_df.to_csv("path_to_folder/quartile_thresholds.csv", index=True)


In [ ]:
#This section computes the weigthed average score per species per polygon, using the quartile scores computed before.

# --- load your scored table ---
df = pd.read_csv("path_to_file/quartile_scores.csv")

id_col = "ASSESS_UNI_NUM" #---polygon ID column

# general layers (will be combined with species data in weighted average) This are layers that are not per-species but are relevant to all species.
general_cols = [
    "in_stream_features_per_km_stream_score",
    "wetland_area_km2_per_km2_ws_score"
]

# 2) species list and container for results
species = ["chinook", "coho", "pink", "steelhead"]
out = df[[id_col]].copy()

# 3) weighted average per species (combined with general columns) All set to 1 here but you can change weights as needed.
for sp in species:
    sp_cols = [c for c in df.columns if c.endswith("_score") and sp in c.lower()]
    w = np.array([1.0 if ("count" in c.lower() or "psf" in c.lower()) else 1.0
                  for c in sp_cols], dtype=float)

    all_cols = sp_cols + general_cols
    X = df[all_cols].copy()
    w = np.append(w, np.ones(len(general_cols)))

    # weighted average per row, ignoring NaNs in both values and weights
    num = X.mul(w, axis=1).sum(axis=1, skipna=True)
    den = (~X.isna()).mul(w, axis=1).sum(axis=1)
    out[f"{sp}_overall_score"] = (num / den)  # <-- This line has the rounding np.round(num / den) if want the rounded number

# 4) save the per-species overall scores. This file has the per-species weighted average scores for each polygon.
out.to_csv("path_to_folder/species_weighted_scores.csv", index=False)

In [ ]:
#To get the final overall score per polygon, you can average the per-species scores as follows:

# --- load the species weighted scores ---
df = pd.read_csv("path_to_folder/species_weighted_scores.csv")

id_col = "ASSESS_UNI_NUM" #---polygon ID column
species = ["chinook", "coho", "pink", "steelhead"]

# Calculate mean of the 4 species scores per ASSESS_UNI_NUM
species_cols = [f"{sp}_overall_score" for sp in species]
df["all_species_score"] = np.round(df[species_cols].mean(axis=1, skipna=True))

# Keep only ID and all_species_score columns
out = df[[id_col, "all_species_score"]].copy()

# Save the result
out.to_csv("path_to_folder/all_species_scores.csv", index=False)

In [ ]:
#Finally, we need to get the polygons score back to the spatial feature. We will have one layer witht the per species and one with the all species as needed.

# ---- EDIT THESE ----
polys_layer_name = "Nicola Sub-watershed"     # polygon layer already in map
csv_path         = r"path_to folder/all_species_scores.csv"   # it can be /species_weighted_scores.csv is you want the per-species scores
out_gdb          = r"path_to_folder\Nicola.gdb"  # geodatabase to write output feature class. This is part of your ArcGIS Pro project. Change name here is your project gdb is different.
out_fc_name      = "allsps_scores"
key = "ASSESS_UNI_NUM" # --- polygon ID field

# -- get layer & make a feature layer we can join to
aprx = arcpy.mp.ArcGISProject("CURRENT"); m = aprx.activeMap
polys_lyr = m.listLayers(polys_layer_name)[0]
arcpy.management.MakeFeatureLayer(polys_lyr, "polys_lyr_view")

# -- bring CSV in as a table (scratch GDB)
scores_tbl = arcpy.CreateUniqueName("scores_tbl", arcpy.env.scratchGDB)
arcpy.conversion.TableToTable(csv_path, arcpy.env.scratchGDB, os.path.basename(scores_tbl))

# -- INNER JOIN: keep only matching ASSESS_UNI_NUM in both datasets
arcpy.management.AddJoin(
    in_layer_or_view="polys_lyr_view",
    in_field=key,
    join_table=scores_tbl,
    join_field=key,
    join_type="KEEP_COMMON"          # <- this enforces "common only"
)

# -- write out a new FC with only the matched polygons + joined fields
out_fc = os.path.join(out_gdb, out_fc_name)
arcpy.management.CopyFeatures("polys_lyr_view", out_fc)

# -- clean up view
arcpy.management.RemoveJoin("polys_lyr_view")

In [ ]:
#This final section is for adding the polygons values to the stream lines. That means that instead of colouring the polygon, we will have the highe order stream lines coloured by the polygon score.

# Get layers from the current ArcGIS Pro project
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.activeMap

# Replace these with the actual layer names in your map
polygons_layer = m.listLayers("allsps_scores")[0]
streams_layer  = m.listLayers("high_order_watershed")[0]

# Output feature class
out_fc = r"path_to_folder\Nicola.gdb\streams_with_poly_attr_allsps_validated" # geodatabase to write output feature class. This is part of your ArcGIS Pro project. Change name here is your project gdb is different.
out_fc_name      = "allsps_scores"

# Intersect: split streams by polygons, keep attributes from both
arcpy.Intersect_analysis(
    in_features=[streams_layer, polygons_layer],
    out_feature_class=out_fc,
    join_attributes="ALL",
    cluster_tolerance="",
    output_type="LINE"
)
